This project explores the global unicorn ecosystem, focusing on companies valued at over $1 billion. The goal is to extract actionable insights for growth strategy, venture capital investment, and industry analysis using SQL.

The analysis follows a structured workflow:

- Data Cleaning: identifying missing values, standardizing industries, filtering for relevant valuations, and ensuring data consistency.

- Core Analysis: examining trends across industries, individual companies, funding by investors, and geographic patterns to uncover key drivers of unicorn creation and growth.

- Conclusions and Recommendations: summarizing strategic insights, highlighting high-performing sectors, and discussing implications for investors and growth-focused decision-making.

By combining rigorous SQL analysis with a growth-oriented perspective, this project demonstrates both technical proficiency and strategic reasoning, providing a portfolio-ready case study suitable for GitHub and interviews in strategy and growth roles.

# 1.Data Quality Assessment
1.1 Duplicate checks
First, I checked whether any companies appeared more than once in the dataset to avoid double-counting unicorns.

In [2]:
-- No duplicates found.
SELECT pc.company_id,
		COUNT(*)
FROM public.companies AS pc
	GROUP BY pc.company_id
HAVING COUNT(*)>1;

,company_id,count


1.2 Geographic consistency (City & Country)
I reviewed city and country fields to identify missing or inconsistent geographic information.
- city nulls → Singapore / Hong Kong
- country GROUP BY → no misspellings detected

In [4]:
-- City analysis
SELECT pc.city,
	pc.country
FROM public.companies AS pc
	WHERE pc.city = '';

,city,country
0,NaN,Singapore
1,NaN,Hong Kong
2,NaN,Singapore
3,NaN,Singapore
4,NaN,Singapore
5,NaN,Singapore
6,NaN,Singapore
7,NaN,Bahamas
8,NaN,Hong Kong
9,NaN,Singapore


In [7]:
-- Country analysis
SELECT pc.country,
	COUNT(*)
FROM public.companies AS pc
GROUP BY pc.country
	ORDER BY COUNT(*) DESC;


,country,count
0,United States,562
1,China,173
2,India,65
3,United Kingdom,43
4,Germany,26
5,France,24
6,Israel,20
7,Canada,19
8,Brazil,16
9,South Korea,12


### **1.3 Missing values by key fields**
I then assessed missing values across key analytical fields to determine which records could be reliably used for different analyses.

In [15]:
-- Company_id (NULL)
SELECT pc.company_id
FROM public.companies AS pc
WHERE pc.company_id IS NULL;

,company_id


In [16]:
-- Company
SELECT pc.company
FROM public.companies AS pc
WHERE pc.company IS NULL
	OR pc.company ='';

,company


In [17]:
-- Valuation
SELECT pf.valuation
FROM public.funding AS pf
WHERE pf.valuation IS NULL;

,valuation


In [2]:
-- Funding
SELECT pf.funding
FROM public.funding AS pf
WHERE pf.funding IS NULL;

,funding
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [18]:
-- Industry
SELECT pi.industry
FROM public.industries AS pi
WHERE pi.industry IS NULL
	OR pi.industry ='';

,industry


In [1]:
-- date
SELECT pd.date_joined, pd.year_founded
	FROM public.dates AS pd
WHERE pd.date_joined IS NULL 
OR year_founded IS NULL;

,date_joined,year_founded


In [6]:
-- date funded > date joined
SELECT pc.company,
	pd.company_id,
	pd.date_joined,
	pd.year_founded,
	(EXTRACT(YEAR FROM pd.date_joined)::numeric-
	pd.year_founded::numeric) AS diff
	FROM public.dates AS pd
	JOIN public.companies AS pc
	ON pd.company_id=pc.company_id
ORDER BY diff
	LIMIT 1;

,company,company_id,date_joined,year_founded,diff
0,Yidian Zixun,714,2017-10-17 00:00:00+00:00,2021,-4


In [5]:
SELECT COUNT(*) AS companies_in_dates_not_in_industries
FROM public.dates d
LEFT JOIN public.industries i
  ON d.company_id = i.company_id
WHERE i.company_id IS NULL;

,companies_in_dates_not_in_industries
0,0


Only one error has been identified in the data for Yidian Zixun: it states that the company was founded in 2021, but the date:joined field shows that it became a unicorn in 2017.


# Core Analysis Overview

After validating data completeness and consistency, the analysis shifts to understanding the dynamics of unicorn creation from a growth and investment perspective.

The objective of this section is not only to describe historical trends, but to extract strategic insights relevant for founders, investors, and growth teams:

- Which industries consistently generate high-value companies
- Whether valuation outcomes are driven by scale (many unicorns) or by outliers
- How investor participation correlates with valuation outcomes
- What patterns emerge across time, sectors, and capital concentration
The analysis combines distributional metrics (mean vs. median), temporal trends, and investor-level aggregation to move beyond descriptive statistics and towards decision-oriented insights.

## **1. Trend Analysis & Market Dynamics**

In [6]:
/*
================================================================================
QUERY 1: Number of unicorns created per year
================================================================================
*/
WITH unicorns AS (
    SELECT DISTINCT pf.company_id
    FROM public.funding AS pf
    WHERE pf.valuation >= 1e9
)
SELECT 
    EXTRACT(YEAR FROM pd.date_joined) AS year,
    COUNT(DISTINCT pi.company_id) AS total_unicorns_year
FROM public.industries AS pi 
JOIN public.dates AS pd ON pi.company_id = pd.company_id
JOIN unicorns AS u ON pi.company_id = u.company_id
GROUP BY year
ORDER BY year DESC;


,year,total_unicorns_year
0,2022,116
1,2021,520
2,2020,108
3,2019,104
4,2018,103
5,2017,44
6,2016,21
7,2015,35
8,2014,13
9,2013,3


In [6]:
/*
================================================================================
QUERY 2: Top 5 Industries by Unicorn Creation per Market Phase
================================================================================
*/

WITH unicorns AS (
    SELECT DISTINCT pf.company_id
    FROM public.funding AS pf
    WHERE pf.valuation >= 1e9
),
base_data AS (
    SELECT 
        pi.company_id,
        pi.industry,
        pd.date_joined,
        CASE 
            WHEN EXTRACT(YEAR FROM pd.date_joined) BETWEEN 2011 AND 2017 THEN '2011–2017 Gradual growth'
            WHEN EXTRACT(YEAR FROM pd.date_joined) BETWEEN 2018 AND 2020 THEN '2018–2020 Strong expansion'
            WHEN EXTRACT(YEAR FROM pd.date_joined) = 2021 THEN '2021 Exceptional boom'
            WHEN EXTRACT(YEAR FROM pd.date_joined) = 2022 THEN '2022 Sharp normalization'
            ELSE 'Other'
        END AS market_phase
    FROM public.industries AS pi
    INNER JOIN public.dates AS pd ON pi.company_id = pd.company_id
    INNER JOIN unicorns ON pi.company_id = unicorns.company_id
    WHERE EXTRACT(YEAR FROM pd.date_joined) >= 2011
),
period_totals AS (
    SELECT 
        market_phase,
        industry,
        COUNT(DISTINCT company_id) AS num_unicorns_period,
        SUM(COUNT(DISTINCT company_id)) OVER (PARTITION BY market_phase) AS total_period_unicorns
    FROM base_data
    GROUP BY market_phase, industry
),
ranked_industries AS (
    SELECT 
        market_phase,
        industry,
        num_unicorns_period,
        ROUND(num_unicorns_period * 100.0 / total_period_unicorns, 2) AS pct_of_period,
        total_period_unicorns,
        ROW_NUMBER() OVER (PARTITION BY market_phase ORDER BY num_unicorns_period DESC) AS industry_rank
    FROM period_totals
),
complete_coverage AS (
	SELECT 
	    market_phase,
	    industry,
	    num_unicorns_period,
	    pct_of_period,
	    total_period_unicorns,
	    industry_rank
	FROM ranked_industries
	WHERE industry_rank <=5
	UNION ALL

	-- 'Rest of Indutries' for rank > 5
	    SELECT 
        market_phase,
        'Rest of Industries' AS industry,
        SUM(num_unicorns_period) AS num_unicorns_period,
        ROUND(SUM(num_unicorns_period) * 100.0 / MAX(total_period_unicorns), 2) AS pct_of_period,
        MAX(total_period_unicorns) AS total_period_unicorns,
        6 AS industry_rank
    FROM ranked_industries
    WHERE industry_rank > 5
    GROUP BY market_phase, total_period_unicorns
)
	SELECT
	    market_phase,
	    industry,
	    num_unicorns_period,
	    pct_of_period,
	    total_period_unicorns
	FROM complete_coverage
ORDER BY 
    CASE market_phase 
        WHEN '2011–2017 Gradual growth' THEN 1
        WHEN '2018–2020 Strong expansion' THEN 2
        WHEN '2021 Exceptional boom' THEN 3
        WHEN '2022 Sharp normalization' THEN 4
        ELSE 5
    END ASC,
    industry_rank ASC;



,market_phase,industry,num_unicorns_period,pct_of_period,total_period_unicorns
0,2011–2017 Gradual growth,E-commerce & direct-to-consumer,24,19.67,122
1,2011–2017 Gradual growth,Mobile & telecommunications,13,10.66,122
2,2011–2017 Gradual growth,Internet software & services,11,9.02,122
3,2011–2017 Gradual growth,Fintech,10,8.20,122
4,2011–2017 Gradual growth,Other,9,7.38,122
5,2011–2017 Gradual growth,Rest of Industries,55,45.08,122
6,2018–2020 Strong expansion,Internet software & services,47,14.92,315
7,2018–2020 Strong expansion,Fintech,45,14.29,315
8,2018–2020 Strong expansion,E-commerce & direct-to-consumer,32,10.16,315
9,2018–2020 Strong expansion,Artificial intelligence,30,9.52,315


In [12]:
/*
================================================================================
QUERY 3: YoY Growth by Industry per Market Phase
          (Top 5 + Rest of Industries)
================================================================================
*/

WITH unicorns AS (
    SELECT DISTINCT pf.company_id
    FROM public.funding AS pf
    WHERE pf.valuation >= 1e9
),
base_data AS (
    SELECT 
        pi.company_id,
        pi.industry,
        pd.date_joined,
        CASE 
            WHEN EXTRACT(YEAR FROM pd.date_joined) BETWEEN 2011 AND 2017 THEN '2011–2017 Gradual growth'
            WHEN EXTRACT(YEAR FROM pd.date_joined) BETWEEN 2018 AND 2020 THEN '2018–2020 Strong expansion'
            WHEN EXTRACT(YEAR FROM pd.date_joined) = 2021 THEN '2021 Exceptional boom'
            WHEN EXTRACT(YEAR FROM pd.date_joined) = 2022 THEN '2022 Sharp normalization'
            ELSE 'Other'
        END AS market_phase,
        EXTRACT(YEAR FROM pd.date_joined) AS year
    FROM public.industries AS pi 
    JOIN public.dates AS pd ON pi.company_id = pd.company_id
    JOIN unicorns AS u ON pi.company_id = u.company_id
    WHERE EXTRACT(YEAR FROM pd.date_joined) >= 2011
),
yearly_counts AS (
    SELECT 
        market_phase,
        industry,
        year,
        COUNT(DISTINCT company_id) AS num_unicorns
    FROM base_data
    GROUP BY market_phase, industry, year
),
yoy_raw AS (
    SELECT 
        market_phase,
        industry,
        year,
        num_unicorns,
        -- LAG in phase
        LAG(num_unicorns) OVER (
            PARTITION BY market_phase, industry 
            ORDER BY year
        ) AS lag_in_phase,
        -- LAG global
        LAG(num_unicorns) OVER (
            PARTITION BY industry 
            ORDER BY year
        ) AS lag_global
    FROM yearly_counts
),
yoy_growth AS (
    SELECT
        market_phase,
        industry,
        year,
        num_unicorns,
        COALESCE(lag_in_phase, lag_global) AS prev_year_unicorns,
        ROUND(
            (num_unicorns - COALESCE(lag_in_phase, lag_global)) * 100.0 
            / NULLIF(COALESCE(lag_in_phase, lag_global), 0),
            2
        ) AS yoy_growth_pct
    FROM yoy_raw
    WHERE COALESCE(lag_in_phase, lag_global) IS NOT NULL
),
ranked AS (
    SELECT
        market_phase,
        year,
        industry,
        num_unicorns,
        yoy_growth_pct,
        ROW_NUMBER() OVER (
            PARTITION BY market_phase 
            ORDER BY num_unicorns DESC, yoy_growth_pct DESC NULLS LAST
        ) AS growth_rank
    FROM yoy_growth
),
top5 AS (
    -- Top 5 industries by phase
    SELECT
        market_phase,
        year,
        industry,
        num_unicorns,
        yoy_growth_pct,
        growth_rank
    FROM ranked
    WHERE growth_rank <= 5
),
rest AS (
    -- "Rest of Industries" by phase
    SELECT
        market_phase,
        NULL::integer AS year,  
        'Rest of Industries' AS industry,
        SUM(num_unicorns) AS num_unicorns,
        NULL::numeric AS yoy_growth_pct,
        6 AS growth_rank
    FROM ranked
    WHERE growth_rank > 5
    GROUP BY market_phase
)
SELECT
    market_phase,
    year,
    industry,
    num_unicorns,
    yoy_growth_pct,
    growth_rank
FROM (
    SELECT * FROM top5
    UNION ALL
    SELECT * FROM rest
) AS final
ORDER BY 
    CASE market_phase 
        WHEN '2011–2017 Gradual growth' THEN 1
        WHEN '2018–2020 Strong expansion' THEN 2
        WHEN '2021 Exceptional boom' THEN 3
        WHEN '2022 Sharp normalization' THEN 4
        ELSE 5
    END,
    growth_rank,
    year DESC NULLS LAST;



,market_phase,year,industry,num_unicorns,yoy_growth_pct,growth_rank
0,2011–2017 Gradual growth,2015.0,E-commerce & direct-to-consumer,9,125.00,1
1,2011–2017 Gradual growth,2017.0,Fintech,6,200.00,2
2,2011–2017 Gradual growth,2016.0,E-commerce & direct-to-consumer,6,-33.33,3
3,2011–2017 Gradual growth,2017.0,Mobile & telecommunications,5,400.00,4
4,2011–2017 Gradual growth,2017.0,Health,4,300.00,5
5,2011–2017 Gradual growth,NaN,Rest of Industries,73,NaN,6
6,2018–2020 Strong expansion,2019.0,Fintech,20,100.00,1
7,2018–2020 Strong expansion,2020.0,Internet software & services,20,53.85,2
8,2018–2020 Strong expansion,2020.0,E-commerce & direct-to-consumer,16,33.33,3
9,2018–2020 Strong expansion,2020.0,Fintech,15,-25.00,4


## **Key Findings**
- Unicorn creation follows a highly cyclical pattern, with gradual growth between 2011–2017, strong expansion in 2018–2020, an exceptional boom in 2021, and a sharp normalization in 2022.

- Fintech and Internet Software & Services emerge as structurally resilient industries, consistently generating unicorns across market cycles and accounting for the largest cumulative share of unicorns over time.

- The market experienced a broad-based acceleration in 2018–2019, driven by diversification into AI, Supply Chain, and deep-tech sectors.

- 2021 represents a clear outlier, with unicorn creation increasing nearly fivefold versus prior years, suggesting funding-driven valuation expansion rather than a sudden structural shift in innovation.

- The subsequent contraction in 2022 indicates a reversion toward sustainable levels, reinforcing the cyclical nature of unicorn formation.

## **2. Valuation & Unit Economics**

In [13]:
/*
================================================================================
QUERY 4: Avg vs Median valuation & Market Cap
Summary by market phase & industry (Top 5 + Rest)
================================================================================
*/

WITH unicorns AS (
    SELECT DISTINCT pf.company_id
    FROM public.funding AS pf
    WHERE pf.valuation >= 1e9
),
base_data AS (
    SELECT 
        pf.company_id,
        pi.industry,
        pf.valuation,
        EXTRACT(YEAR FROM pd.date_joined) AS year,
        CASE 
            WHEN EXTRACT(YEAR FROM pd.date_joined) BETWEEN 2011 AND 2017 THEN '2011 - 2017 Gradual Growth'
            WHEN EXTRACT(YEAR FROM pd.date_joined) BETWEEN 2018 AND 2020 THEN '2018 - 2020 Strong expansion'
            WHEN EXTRACT(YEAR FROM pd.date_joined) = 2021 THEN '2021 Exceptional boom'
            WHEN EXTRACT(YEAR FROM pd.date_joined) = 2022 THEN '2022 Sharp normalization'
            ELSE 'Other'
        END AS market_phase
    FROM public.funding AS pf
    JOIN unicorns u ON pf.company_id = u.company_id
    LEFT JOIN public.industries AS pi ON pf.company_id = pi.company_id
    LEFT JOIN public.dates AS pd ON pf.company_id = pd.company_id
),
stats_by_phase AS (
    SELECT
        market_phase,
        industry,
        COUNT(DISTINCT company_id) AS num_companies,
        SUM(valuation) / 1e9::numeric AS market_cap_bn,
        AVG(valuation) / 1e9::numeric AS avg_valuation_bn,
        PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY valuation)::numeric / 1e9::numeric AS median_valuation_bn
    FROM base_data
    WHERE market_phase <> 'Other'
    GROUP BY market_phase, industry
),
stats_with_skew AS (
    SELECT
        market_phase,
        industry,
        num_companies,
        ROUND(market_cap_bn::numeric, 2) AS market_cap_bn,
        ROUND(avg_valuation_bn::numeric, 2) AS avg_valuation_bn,
        ROUND(median_valuation_bn::numeric, 2) AS median_valuation_bn,
        ROUND((avg_valuation_bn - median_valuation_bn)::numeric, 2) AS diff_bn,
        ROUND(
            ((avg_valuation_bn - median_valuation_bn)
             / NULLIF(median_valuation_bn, 0))::numeric * 100, 2
        ) AS skew_pct
    FROM stats_by_phase
),
ranked AS (
    SELECT
        market_phase,
        industry,
        num_companies,
        market_cap_bn,
        avg_valuation_bn,
        median_valuation_bn,
        diff_bn,
        skew_pct,
        ROW_NUMBER() OVER (
            PARTITION BY market_phase
            ORDER BY market_cap_bn DESC
        ) AS cap_rank
    FROM stats_with_skew
),
top5 AS (
    SELECT *
    FROM ranked
    WHERE cap_rank <= 5
),
rest AS (
    SELECT
        market_phase,
        'Rest of Industries' AS industry,
        SUM(num_companies) AS num_companies,
        ROUND(SUM(market_cap_bn)::numeric, 2) AS market_cap_bn,
        NULL::numeric AS avg_valuation_bn,
        NULL::numeric AS median_valuation_bn,
        NULL::numeric AS diff_bn,
        NULL::numeric AS skew_pct,
        6 AS cap_rank
    FROM ranked
    WHERE cap_rank > 5
    GROUP BY market_phase
)
SELECT
    market_phase,
    industry,
    num_companies,
    market_cap_bn,
    avg_valuation_bn,
    median_valuation_bn,
    diff_bn,
    skew_pct
FROM (
    SELECT * FROM top5
    UNION ALL
    SELECT * FROM rest
) f
ORDER BY 
    CASE market_phase 
        WHEN '2011 - 2017 Gradual Growth' THEN 1
        WHEN '2018 - 2020 Strong expansion' THEN 2
        WHEN '2021 Exceptional boom' THEN 3
        WHEN '2022 Sharp normalization' THEN 4
        ELSE 5
    END,
    cap_rank;



,market_phase,industry,num_companies,market_cap_bn,avg_valuation_bn,median_valuation_bn,diff_bn,skew_pct
0,2011 - 2017 Gradual Growth,Artificial intelligence,8,196.0,24.50,2.0,22.50,1125.00
1,2011 - 2017 Gradual Growth,Fintech,10,162.0,16.20,2.0,14.20,710.00
2,2011 - 2017 Gradual Growth,Other,9,113.0,12.56,1.0,11.56,1155.56
3,2011 - 2017 Gradual Growth,E-commerce & direct-to-consumer,24,94.0,3.92,1.0,2.92,291.67
4,2011 - 2017 Gradual Growth,Edtech,8,52.0,6.50,2.5,4.00,160.00
5,2011 - 2017 Gradual Growth,Rest of Industries,63,256.0,NaN,NaN,NaN,NaN
6,2018 - 2020 Strong expansion,Fintech,45,287.0,6.38,4.0,2.38,59.44
7,2018 - 2020 Strong expansion,Internet software & services,47,253.0,5.38,4.0,1.38,34.57
8,2018 - 2020 Strong expansion,E-commerce & direct-to-consumer,32,204.0,6.38,2.0,4.38,218.75
9,2018 - 2020 Strong expansion,Artificial intelligence,30,114.0,3.80,3.0,0.80,26.67


## **1. Average vs Median Valuation (Skewness Analysis)**

**Key Findings**

Across phases and industries, average valuations systematically sit above median valuations, revealing right‑skewed distributions driven by a handful of very large outliers.

This skewness is especially visible in the 2011–2017 gradual growth phase, where a relatively small base of unicorns combined with a few very large companies pulls the average up and widens the gap versus the median, indicating that valuation expansion disproportionately benefited the very top of the distribution.

Fintech, Internet Software & Services, Artificial Intelligence, and the Other vertical display the largest and most persistent gaps between average and median valuations, reinforcing a winner‑takes‑most pattern within these segments, particularly in the early phase.

Median valuations remain comparatively stable over time, which suggests that headline valuation growth is driven more by outliers than by broad-based improvements across the typical unicorn.

### 2. Market Capitalization by Industry ###    
**Key Findings**

Market capitalization is heavily concentrated in a few core industries, with Fintech and Internet Software & Services capturing the largest share of total unicorn market cap.

These sectors combine high company counts with elevated average valuations, positioning them as structural engines of the unicorn landscape rather than cyclical winners.

Other verticals such as Artificial Intelligence and E‑commerce contribute significantly to the number of unicorns but concentrate less capital, pointing either to earlier-stage sector maturity or to tighter valuation ceilings relative to the core leaders.


In [2]:
/*
================================================================================
QUERY 5: Valuation Band Analysis (1-2bn, 2-5bn, 5-10bn, +10bn)
================================================================================
*/

SELECT pi.industry,
	COUNT(*) AS total_unicorns,
SUM(CASE WHEN pf.valuation BETWEEN 1e9  AND 2e9 THEN 1 ELSE 0 END) AS tier_1_2B,
SUM (CASE WHEN pf.valuation BETWEEN 2e9 AND 5e9 THEN 1 ELSE 0 END) AS tier_2_5B,
SUM(CASE WHEN pf.valuation BETWEEN  5e9 AND 10e9 THEN 1 ELSE 0 END)AS tier_5_10B,
SUM(CASE WHEN pf.valuation>10e9 THEN 1 ELSE 0 END) AS tier_Over_10B,
	ROUND(AVG (pf.valuation)/1e9,2) AS avg_valuation_bn,
	ROUND(SUM(pf.valuation)/1e9,2) AS total_market_cap_bn
FROM public.funding AS pf
JOIN public.industries AS pi ON pf.company_id = pi.company_id
WHERE pf.valuation >=1e9
GROUP BY pi.industry
ORDER BY total_market_cap_bn DESC;



,industry,total_unicorns,tier_1_2b,tier_2_5b,tier_5_10b,tier_over_10b,avg_valuation_bn,total_market_cap_bn
0,Fintech,224,144,101,33,11,3.94,882.0
1,Internet software & services,205,134,96,21,6,2.90,595.0
2,E-commerce & direct-to-consumer,111,73,48,8,7,3.84,426.0
3,Artificial intelligence,84,63,36,9,3,4.49,377.0
4,Other,58,44,25,3,3,4.34,252.0
5,Health,74,49,29,11,2,2.68,198.0
6,"Supply chain, logistics, & delivery",57,42,20,4,3,3.11,177.0
7,Data management & analytics,41,27,22,4,2,3.32,136.0
8,Cybersecurity,50,32,22,7,0,2.58,129.0
9,Consumer & retail,25,20,9,2,3,4.24,106.0


## **3. Valuation Band (Tier) Analysis**

**Key Findings**

- Across all major industries, the majority of unicorns are concentrated in the €1–2B and €2–5B valuation tiers, confirming that extreme valuations remain relatively rare.

- Fintech and Internet Software & Services not only produce the most unicorns, but also the highest number of companies in upper valuation tiers (€5–10B+), reinforcing their dominance in generating large-scale outcomes.

- Industries such as Artificial Intelligence and E-commerce show similar tier distributions, but with fewer companies breaking into the highest valuation bands, suggesting a more constrained scaling profile.

- Industry leadership is driven not just by unicorn counts, but by the ability to consistently generate upper-tier outcomes.

## **3. Market Concentration & **Competition****

In [3]:
/*
================================================================================
QUERY 6: Top 5 industries by market cap + Herfindahl Index
================================================================================
*/

WITH industry_caps AS( 
	SELECT pi.industry,
	ROUND(SUM(pf.valuation)/1e9,2) AS market_cap_bn,
	SUM(pf.valuation)/ SUM(SUM(pf.valuation)) OVER() AS market_share
	FROM public.funding AS pf
	JOIN public.industries AS pi
	ON pf.company_id = pi.company_id
	WHERE pf.valuation>=1e9
	GROUP BY pi.industry
)
SELECT industry, 
	market_cap_bn,
	ROUND(market_share * 100, 2) AS market_share_ptc,
	ROUND(POWER(market_share, 2) * 10000, 2) AS hhi_contribution,
	RANK() OVER (ORDER BY market_cap_bn DESC) AS rank_by_cap
FROM industry_caps
ORDER BY market_cap_bn DESC
	LIMIT 5;


,industry,market_cap_bn,market_share_ptc,hhi_contribution,rank_by_cap
0,Fintech,882.0,23.77,564.88,1
1,Internet software & services,595.0,16.03,257.07,2
2,E-commerce & direct-to-consumer,426.0,11.48,131.78,3
3,Artificial intelligence,377.0,10.16,103.21,4
4,Other,252.0,6.79,46.11,5


## **Key Findings**
- The unicorn market is highly concentrated across a small number of industries, with Fintech clearly emerging as the dominant sector, accounting for approximately 23% of total unicorn market capitalization.

- Internet Software & Services ranks second, representing around 16% of total market cap, followed by E-commerce (11%) and Artificial Intelligence (10%).

- Together, the top four industries account for over 60% of total unicorn market capitalization, indicating a meaningful concentration of value despite a broad set of active sectors.

- Fintech exhibits the highest individual contribution to the Herfindahl-Hirschman Index (HHI), signaling that not only is the sector large, but it also plays a disproportionate role in driving overall market concentration.

- Beyond the top industries, HHI contributions decline sharply, suggesting a long tail of fragmented sectors with limited impact on total market concentration.

## **Strategic Interpretation**

- The unicorn ecosystem exhibits a hybrid structure:
a small number of capital-dominant industries coexisting with a fragmented long tail of niche sectors.

- Fintech and Internet Software & Services function as “core allocation sectors”, where capital concentration, scale, and repeatability of large outcomes are highest.

- In contrast, smaller industries contribute to diversification but offer lower systemic impact on aggregate market value.

- From an investment perspective, this implies that portfolio-level exposure to unicorn outcomes is likely driven primarily by sector allocation rather than by marginal diversification across smaller industries.

## **4. Cohort & Velocity Analysis**

In [7]:
/*
================================================================================
QUERY 7: Velocity on becoming a Unicorn  by industry
================================================================================
*/
SELECT pi.industry,
	COUNT(*) AS number_unicorns,
	ROUND(
	 AVG(
	  EXTRACT(YEAR FROM pd.date_joined)::numeric - 
	pd.year_founded::numeric
	),
	2) AS avg_years_to_unicorn,
	MIN(
	EXTRACT(YEAR FROM pd.date_joined)::integer - 
	pd.year_founded::integer
	) AS fastest_years,
	MAX(
	EXTRACT(YEAR FROM pd.date_joined)::integer- 
	pd.year_founded::integer
	)AS slowest_years
FROM public.funding AS pf
JOIN public.industries AS pi
ON pf.company_id = pi.company_id
JOIN public.dates AS pd
ON pf.company_id = pd.company_id
WHERE pf.valuation>=1e9 
	AND pd.year_founded IS NOT NULL
	AND pd.year_founded <=EXTRACT(YEAR FROM pd.date_joined)-- excluir error identificado
GROUP BY pi.industry
HAVING COUNT(*)>3 --Only industries with +3 unicorns
ORDER BY avg_years_to_unicorn;

,industry,number_unicorns,avg_years_to_unicorn,fastest_years,slowest_years
0,Auto & transportation,31,5.03,0,14
1,Artificial intelligence,84,5.89,0,15
2,Hardware,34,5.94,1,18
3,E-commerce & direct-to-consumer,111,6.43,0,24
4,Fintech,224,6.54,0,27
5,Travel,14,6.57,3,14
6,Mobile & telecommunications,37,6.59,1,20
7,Cybersecurity,50,6.80,1,21
8,"Supply chain, logistics, & delivery",57,6.93,2,20
9,Edtech,28,7.71,3,17


## **Key Findings**

- The time required for a startup to reach unicorn status varies materially across industries, indicating meaningful differences in scaling dynamics.

- Auto & Transportation emerges as the fastest-moving sector, with an average time to unicorn of approximately 5 years, followed closely by Artificial Intelligence (5.9 years) and Hardware (5.9 years).

- Core digital sectors such as E-commerce (6.4 years) and Fintech (6.5 years) exhibit moderate time-to-scale, reflecting a balance between rapid adoption and regulatory or operational complexity.

- Consumer-facing and healthcare-related industries show the slowest paths to unicorn status, with average timelines exceeding 8 years, suggesting longer product validation cycles and higher structural friction.

- Overall, the gap between the fastest and slowest industries is approximately three years, highlighting that industry selection materially affects scaling velocity.

## **Strategic Interpretation**

- Faster time-to-unicorn industries tend to be capital-intensive or technology-driven, where scale advantages and network effects can be realized earlier.

- Slower industries likely face longer go-to-market cycles, regulatory constraints, or operational complexity, which delay value realization despite potentially large long-term outcomes.

- From a growth or investment perspective, time-to-scale should be evaluated alongside valuation potential, as faster velocity can significantly improve capital efficiency and portfolio turnover.

## **5. Geographic & Funding Patterns**

In [8]:
/*
================================================================================
QUERY 8: Unicorn location by country
================================================================================
*/

SELECT pc.country, 
	pc.city, 
	COUNT(*) AS num_unicorns,
	ROUND(SUM(pf.valuation)/1e9,2) AS total_valuation_bn,
	ROUND(AVG(pf.valuation)/1e9,2) AS avg_valuation_bn,
	STRING_AGG(DISTINCT pi.industry, ' ,') AS top_industries
FROM public.companies AS pc
	JOIN public.funding AS pf
	ON pc.company_id = pf.company_id
	JOIN public.industries AS pi
	ON pc.company_id = pi.company_id
	WHERE pf.valuation >=1e9
	GROUP BY country, city
	HAVING COUNT(*)>2 
	ORDER BY total_valuation_bn DESC 
	LIMIT 20;


,country,city,num_unicorns,total_valuation_bn,avg_valuation_bn,top_industries
0,United States,San Francisco,152,724.0,4.76,"Artificial intelligence ,Consumer & retail ,Cy..."
1,China,Beijing,63,348.0,5.52,"Artificial intelligence ,Consumer & retail ,Da..."
2,United States,New York,103,233.0,2.26,"Artificial intelligence ,Auto & transportation..."
3,United Kingdom,London,34,178.0,5.24,"Artificial intelligence ,Auto & transportation..."
4,China,Shenzhen,19,141.0,7.42,"Artificial intelligence ,E-commerce & direct-t..."
5,India,Bengaluru,29,107.0,3.69,"Auto & transportation ,E-commerce & direct-to-..."
6,China,Shanghai,44,99.0,2.25,"Artificial intelligence ,Auto & transportation..."
7,United States,Boston,16,69.0,4.31,"Artificial intelligence ,Cybersecurity ,Data m..."
8,Sweden,Stockholm,6,63.0,10.50,"Auto & transportation ,Fintech ,Health ,Intern..."
9,United States,Palo Alto,18,53.0,2.94,"Artificial intelligence ,Cybersecurity ,Data m..."


In [1]:
/*
================================================================================
QUERY 9: Capital efficiency: valuation vs total capital raised
================================================================================
*/

SELECT 
	pi.industry, 
	COUNT(*) AS num_unicorns,
	ROUND(AVG(pf.funding)::numeric/1e6,2) AS avg_funding_raised_m,
	ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY pf.funding)::numeric/1e6,2) AS median_funding_raised_m,
	ROUND((AVG(pf.valuation)::numeric) / 1e9,2) AS avg_valuation_bn, 
	ROUND(AVG(pf.valuation / NULLIF(pf.funding,0))::numeric,2) AS avg_valuation_multiple
FROM public.funding as pf
JOIN public.industries AS pi
ON pf.company_id = pi.company_id
WHERE 
	pf.valuation>=1e9
	AND pf.funding IS NOT NULL
	AND pf.funding >0
GROUP BY pi.industry
HAVING COUNT(*)>3
ORDER BY avg_valuation_multiple DESC;

,industry,num_unicorns,avg_funding_raised_m,median_funding_raised_m,avg_valuation_bn,avg_valuation_multiple
0,Internet software & services,203,359.38,307.0,2.92,28.30
1,Other,56,715.07,347.5,4.45,8.98
2,Mobile & telecommunications,37,402.27,310.0,2.38,8.92
3,Fintech,222,486.47,370.0,3.96,8.72
4,Hardware,34,628.97,373.5,2.91,7.59
5,Consumer & retail,24,1062.17,431.5,4.38,7.50
6,Data management & analytics,40,449.53,346.5,3.38,6.25
7,Artificial intelligence,84,557.17,302.5,4.49,6.00
8,Health,73,464.99,373.0,2.66,5.82
9,E-commerce & direct-to-consumer,109,682.91,471.0,3.86,5.63


## **Key Findings**

- Internet Software & Services is a clear outlier in capital efficiency, with an average valuation multiple of 28.3x, far exceeding all other industries.

This reflects high scalability, low marginal costs, and strong network effects, allowing companies to reach unicorn status with comparatively limited capital.

- Notably, this industry combines high capital efficiency with scale, as it also has one of the highest unicorn counts (203).

- Fintech, Mobile & Telecommunications, and “Other” form a second efficiency tier, with valuation multiples clustered around 8–9x.

- Fintech stands out by combining high efficiency (8.7x) with the largest number of unicorns (222), reinforcing its role as both a volume and value driver in the ecosystem.

- AI, Data & Analytics, and Consumer-facing sectors show moderate efficiency, with multiples in the 5–6x range.

Despite strong headline valuations (AI averages ~4.5bn), these sectors require materially higher capital investment, reflecting compute intensity, talent costs, and longer paths to monetization.

- Capital-intensive industries underperform on efficiency, despite achieving unicorn scale:

- Auto & Transportation, Supply Chain, and Travel exhibit the lowest multiples (3–4x), requiring €800m–€1.1bn on average to reach valuations similar to software-led sectors.

These industries face structural constraints, including asset intensity, regulation, and operational complexity, which dilute capital productivity.

## Strategic Interpretation

- Valuation alone is not a sufficient indicator of investment attractiveness; capital efficiency varies by nearly 10x across industries.

The data highlights a structural trade-off:

- Software-native sectors optimize for capital efficiency and speed.

- Physical and logistics-heavy sectors optimize for scale and defensibility, at the cost of higher capital consumption.

From an investor or strategy perspective, Fintech and Internet Software offer the strongest risk-adjusted profiles, balancing scale, efficiency, and ecosystem depth.

## **6. Advanced - Predictive indicators**

In [2]:
/*
================================================================================
QUERY 10: Industry Momentum Score
================================================================================
*/ 

WITH metrics AS (
    SELECT 
        pi.industry,
        COUNT(*) AS total_unicorns,
        SUM(pf.valuation) / 1e9 AS total_market_cap_bn,
        AVG(pf.valuation) / 1e9 AS avg_valuation_bn,
        SUM(
            CASE 
                WHEN EXTRACT(YEAR FROM pd.date_joined) >= 2020 
                THEN 1 
                ELSE 0 
            END
        ) AS recent_unicorns
    FROM public.funding AS pf
    JOIN public.industries AS pi
        ON pf.company_id = pi.company_id
    JOIN public.dates AS pd
        ON pf.company_id = pd.company_id
    WHERE pf.valuation >= 1e9
    GROUP BY pi.industry
),
scored AS (
    SELECT
        industry,
        total_unicorns,
        CAST(total_market_cap_bn AS numeric) AS total_market_cap_bn,
        CAST(avg_valuation_bn AS numeric) AS avg_valuation_bn,
        recent_unicorns,
        (COALESCE(recent_unicorns::numeric / NULLIF(total_unicorns::numeric, 0), 0) * 40) AS recent_component,
        (PERCENT_RANK() OVER (ORDER BY total_market_cap_bn) * 30)::numeric AS cap_component,
        (PERCENT_RANK() OVER (ORDER BY avg_valuation_bn) * 30)::numeric AS val_component
    FROM metrics
    WHERE total_unicorns >= 3
)
SELECT
    industry,
    total_unicorns,
    ROUND(total_market_cap_bn, 2) AS total_market_cap_bn,
    ROUND(avg_valuation_bn, 2) AS avg_valuation_bn,
    recent_unicorns,
    ROUND(recent_component + cap_component + val_component, 2) AS momentum_score
FROM scored
ORDER BY momentum_score DESC;



,industry,total_unicorns,total_market_cap_bn,avg_valuation_bn,recent_unicorns,momentum_score
0,Fintech,224,882.0,3.94,184,86.43
1,Artificial intelligence,84,377.0,4.49,49,76.90
2,Other,58,252.0,4.34,37,74.80
3,E-commerce & direct-to-consumer,111,426.0,3.84,70,72.37
4,Internet software & services,205,595.0,2.90,167,66.87
5,Data management & analytics,41,136.0,3.32,31,62.39
6,Health,74,198.0,2.68,56,53.84
7,"Supply chain, logistics, & delivery",57,177.0,3.11,35,52.42
8,Edtech,28,100.0,3.57,17,52.14
9,Consumer & retail,25,106.0,4.24,8,49.23


## **Key Finding**

1. Fintech domina con momentum máximo (86.43)
Fintech lidera claramente con 224 unicorns totales, $882BN de market cap y 184 unicorns recientes (82% de su total desde 2020). Representa el sector más "hot" con combinación perfecta de tamaño, recencia y valuaciones sólidas ($3.94BN promedio).
​

2. Artificial Intelligence emerge como #2 (76.90)
AI muestra alto momentum gracias a valuaciones medias más altas ($4.49BN vs $3.94BN de Fintech) y 58% de unicorns recientes. Aunque menor en tamaño absoluto, su calidad de valuación lo posiciona como sector de alto crecimiento.
​

3. E-commerce D2C mantiene fuerza (72.37)
111 unicorns con 63% recientes confirman que el modelo direct-to-consumer sigue siendo relevante, aunque con menor intensidad de momentum que Fintech/AI.
​

4. Sectores "commodity" pierden momentum
Hardware (32.86), Auto/Transportation (31.34) y Travel (26.43) muestran bajo momentum debido a baja recencia (<50% unicorns post-2020) y menor apetito de inversores.
​

Implicaciones Estratégicas para Growth Teams
Priorizar Fintech y AI: Máximo momentum combina tamaño + recencia + valuaciones premium. Ideales para partnerships, hiring y M&A.
​

Monitorear "Other" (74.80): Categoría catch-all con alto momentum sugiere aparición de nuevas verticales no clasificadas aún.

Evitar sectores maduros: Hardware, Travel y Mobile Telecom muestran bajo momentum, señal de menor potencial de crecimiento exponencial
Internet Software & Services rankea alto en recencia (81%) y market cap (2do lugar), aunque penalizado ligeramente por menor valoración media. Su volumen absoluto lo posiciona como actor clave.


## **Project Conclusion**

This project provides a structured, data-driven overview of the global unicorn ecosystem, combining temporal dynamics, valuation mechanics, competitive concentration, capital efficiency, geographic clustering, and forward-looking momentum indicators.

## **Key Structural Insights**

Fintech stands out as the backbone of the unicorn ecosystem, consistently present across all time periods, leading in total unicorn count, market capitalization, and industry momentum. Its ability to combine scale, capital efficiency, and sustained post-2020 growth positions it as the most structurally resilient sector.

Internet Software & Services emerges as the most capital-efficient industry, achieving the highest valuation multiples by a wide margin. Despite lower average valuations, its massive scale and high recency of unicorn creation confirm it as a long-term growth engine driven by software scalability and network effects.

Artificial Intelligence shows strong forward momentum, ranking high in average valuations and time-to-unicorn speed. While smaller in absolute size, its valuation quality and recent acceleration indicate a sector transitioning from early adoption to structural relevance.

Market concentration remains moderate, with no single industry dominating excessively, suggesting a diversified innovation landscape rather than a winner-takes-all dynamic. Fintech leads in market share, but competitive intensity remains distributed across multiple sectors.

Capital efficiency varies dramatically by industry, with valuation multiples ranging from ~3x in asset-heavy sectors (Auto, Supply Chain, Travel) to over 28x in software-driven industries. This highlights a clear trade-off between scalability and capital intensity.

Geographically, the ecosystem is highly concentrated, led by the United States (San Francisco and New York) and China (Beijing, Shenzhen). However, secondary hubs in Europe, India, Israel, and Southeast Asia signal increasing global diversification.

## **Forward-Looking Perspective**

The Industry Momentum Score reinforces these findings by identifying sectors where recent unicorn creation, valuation quality, and scale converge. Fintech and AI clearly dominate near-term growth potential, while Internet Software & Services continues to expand at scale despite maturity signals.

## **Final Takeaway**

Unicorn creation is not driven solely by valuation size or funding volume, but by a combination of capital efficiency, speed to scale, and sustained industry momentum. Software-native and fintech-driven sectors consistently outperform capital-intensive industries, both historically and predictively.